# Azure AI Safety Multi-modal Evaluations

## Objective

This tutorial demonstrates the evaluation of quality and safety evaluations for following multi-modal (text + images) scenarios.

This tutorial uses the following Azure AI services:

- [Azure AI Safety Evaluation](https://aka.ms/azureaistudiosafetyeval)
- [azure-ai-evaluation](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk)

## Time

You should expect to spend 15 minutes running this sample. 

## About this example

This example demonstrates running quality and safety evaluation on multi-modal (text + image) datasets. 

## Before you begin

### Prerequesite
Have an azure ai project in regions that support the harmful evaluation. More information [here](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk#region-support)


Install the following packages required to execute this notebook. 

In [ ]:
%pip install azure-ai-evaluation --upgrade

### Parameters and imports

In [ ]:
from pprint import pprint
from azure.ai.evaluation import ViolenceEvaluator, SexualEvaluator, SelfHarmEvaluator, HateUnfairnessEvaluator
from azure.identity import DefaultAzureCredential

azure_ai_project = {
    "subscription_id": "<your-subscription-id>",
    "resource_group_name": "<your-resource-group>",
    "project_name": "<your-project-name>",
}

### 1. Multi-modal Content Safety Evaluator

In [ ]:
violence_evaluator = ViolenceEvaluator(credential=DefaultAzureCredential(), azure_ai_project=azure_ai_project)
sexual_evaluator = SexualEvaluator(credential=DefaultAzureCredential(), azure_ai_project=azure_ai_project)
self_harm_evaluator = SelfHarmEvaluator(credential=DefaultAzureCredential(), azure_ai_project=azure_ai_project)
hate_unfair_evaluator = HateUnfairnessEvaluator(credential=DefaultAzureCredential(), azure_ai_project=azure_ai_project)

conversation = {
    "messages": [
        {
            "role": "system",
            "content": [{"type": "text", "text": "You are an AI Assistant that can describe images"}],
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Can you describe this image?"},
                {
                    "type": "image_url",
                    "image_url": {"url": "https://cdn.britannica.com/68/178268-050-5B4E7FB6/Tom-Cruise-2013.jpg"},
                },
            ],
        },
        {
            "role": "assistant",
            "content": [
                {
                    "type": "text",
                    "text": "The image shows a man with short brown hair smiling, wearing a dark-colored shirt.",
                }
            ],
        },
    ]
}

result = violence_evaluator(conversation=conversation)
pprint(result)
result = sexual_evaluator(conversation=conversation)
pprint(result)
result = self_harm_evaluator(conversation=conversation)
pprint(result)
result = hate_unfair_evaluator(conversation=conversation)
pprint(result)

### 2. Content Safety Evaluator supports multi-modal images + text
Following code can run all the above individual safety evaluator together in one composite evaluator called Content Safety Evaluator.

In [ ]:
from pprint import pprint
from azure.ai.evaluation import ContentSafetyEvaluator

evaluator = ContentSafetyEvaluator(credential=DefaultAzureCredential(), azure_ai_project=azure_ai_project)
result = evaluator(conversation=conversation)
pprint(result)

### 3. Protected Material Evaluator supports multi-modal images + text

In [ ]:
from pprint import pprint
from azure.ai.evaluation import ProtectedMaterialEvaluator

evaluator = ProtectedMaterialEvaluator(credential=DefaultAzureCredential(), azure_ai_project=azure_ai_project)
result = evaluator(conversation=conversation)
pprint(result)

### 4. Using Evaluate API

In [ ]:
import pathlib


file_path = pathlib.Path("data.jsonl")

from azure.ai.evaluation import evaluate

content_safety_eval = ContentSafetyEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())

result = evaluate(
    data=file_path,
    azure_ai_project=azure_ai_project,
    evaluators={"content_safety": content_safety_eval},
)
pprint(result)

### 5. base 64 encoded images
Here's how a conversation looks like when you want to run evaluations on an image with b64 encoding

In [ ]:
from pathlib import Path
import base64

base64_image = ""

with Path("image1.jpg").open("rb") as image_file:
    base64_image = base64.b64encode(image_file.read()).decode("utf-8")

conversation = {
    "messages": [
        {"content": "create an image of a branded apple", "role": "user"},
        {
            "content": [{"type": "image_url", "image_url": {"url": f"data:image/jpg;base64,{base64_image}"}}],
            "role": "assistant",
        },
    ]
}
result = evaluator(conversation=conversation)
pprint(result)